# SageMaker와 RoboMaker로 분산 DeepRacer RL 훈련

---
## 소개


이 노트북에서는 Amazon SageMaker RL 및 AWS RoboMaker의 3D 운전 시뮬레이터를 사용하여 강화 학습을 사용하여 완전 자율 1/18 스케일 경주 용 자동차를 훈련시킵니다. [AWS RoboMaker](https://console.aws.amazon.com/robomaker/home#welcome) 는 개발자가 로봇 공학 애플리케이션을 쉽게 개발, 테스트 및 배포할 수 있는 서비스입니다.

이 노트북은 [AWS DeepRacer](https://console.aws.amazon.com/deepracer/home#welcome)의 탈옥 경험을 제공하여 훈련/시뮬레이션 프로세스 및 RL 알고리즘 튜닝을 보다 강력하게 제어합니다.

![Training in Action](./deepracer-reinvent-track.jpg)


---
## 어떻게 동작하나요?

![How training works](./training.png)

강화 학습 에이전트(즉, 자율 주행차)는 보상 기댓값(expected rward)을 극대화하기 위해 주어진 상태(state)에서 행동(action)를 취함으로써 환경과의 상호 작용(예: 트랙)을 통해 운전하는 법을 배웁니다. 에이전트는 반복되는 에피소드를 통해 시행 착오(trial-and-error)를 통해 훈련에서 최적의 행동 계획을 학습합니다.
  
위 그림은 SageMaker 및 **롤아웃**을 수행하는 두 개의 RoboMaker 시뮬레이션 환경에서 분산 RL 훈련의 예시를 보여줍니다. 현재 모델 또는 정책(policy)을 사용하여 고정된 수의 에피소드를 실행합니다. 롤아웃은 에이전트 경험(상태 전이 튜플)을 수집하고 이 데이터를 SageMaker와 공유하여 훈련합니다. SageMaker는 모델 정책을 업데이트한 후, 다음 롤아웃 시퀀스를 실행하는 데 사용됩니다. 이 훈련 루프는 모델이 수렴할 때까지, 즉 자동차가 운전하는 법을 배우고 트랙을 벗어나는 것을 멈출 때까지 계속됩니다. 보다 엄밀하게, 다음과 같은 관점에서 문제를 정의할 수 있습니다.

1. **목표(Objective)**: 트랙 중앙에 가까이 머물면서 자율적으로 운전하는 법을 배웁니다.
2. **환경(Environment)**: AWS RoboMaker에서 호스팅되는 3D 구동 시뮬레이터
3. **상태(State)**: 위 그림과 같이 자동차 헤드 카메라로 캡처한 주행 POV 이미지
4. **행동(Action)**: 다른 각도에서 6개의 이산(discrete) 스티어링 휠 위치 (구성 가능)
5. **보상(Reward)**: 중심선(center line)에 가깝게 머무르는 것에 대한 긍정적인 보상; 오프 트랙에 대한 높은 페널티. 보상 함수는 구성 가능하며 더 복잡하게 만들 수도 있습니다. (예: steering 페널티 추가 가능)

## 전제 조건(Pre-requisites)

### 시뮬레이션 응용 프로그램을 변경하려는 경우에만 아래 명령을 사용하세요 (RoboMaker 코드 변경).

In [ ]:
# #
# # Run these commands if you want to modify the simapp
# #
# # Clean the build directory if present
# !python3 sim_app_bundler.py --clean

# # Download Robomaker simApp from the deepracer public s3 bucket
# simulation_application_bundle_location = "s3://deepracer-managed-resources-us-east-1/deepracer-simapp.tar.gz"
# !aws s3 cp {simulation_application_bundle_location} ./

# # Untar the simapp bundle
# !python3 sim_app_bundler.py --untar ./deepracer-simapp.tar.gz

# # Now modify the simapp from build directory and run this command.

# # Most of the simapp files can be found here (Robomaker changes)
# # bundle/opt/install/sagemaker_rl_agent/lib/python3.5/site-packages/
# # bundle/opt/install/deepracer_simulation_environment/share/deepracer_simulation_environment/
# # bundle/opt/install/deepracer_simulation_environment/lib/deepracer_simulation_environment/

# # # Copying the notebook src/markov changes to the simapp (For sagemaker container)
# !rsync -av ./src/markov/ ./build/simapp/bundle/opt/install/sagemaker_rl_agent/lib/python3.5/site-packages/markov

# !python3 sim_app_bundler.py --tar

### 라이브러리 임포트

시작하기 위해, 필요한 Python 라이브러리를 가져와서 권한 및 구성을 위한 몇 가지 전제 조건으로 환경을 설정합니다.

로컬 시스템 또는 SageMaker 노트북 인스턴스에서 이 노트북을 실행할 수 있습니다. 이 두 시나리오에서 다음 코드 셀들을 실행하여 SageMaker에서 훈련 작업을 시작하고 RoboMaker에서 시뮬레이션 작업을 시작할 수 있습니다.

In [ ]:
import boto3
import sagemaker
import sys
import os
import re
import numpy as np
import subprocess
sys.path.append("common")
from misc import get_execution_role, wait_for_s3_object
from docker_utils import build_and_push_docker_image
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
from time import gmtime, strftime
import time
from IPython.display import Markdown
from markdown_helper import *

### 기본 파라메터 초기화

In [ ]:
# Select the instance type
instance_type = "ml.c4.2xlarge"
#instance_type = "ml.p2.xlarge"
#instance_type = "ml.c5.4xlarge"

# Starting SageMaker session
sage_session = sagemaker.session.Session()

# Create unique job name.
job_name_prefix = 'deepracer-notebook'

# Duration of job in seconds (1 hours)
job_duration_in_seconds = 3600

# AWS Region
aws_region = sage_session.boto_region_name
if aws_region not in ["us-west-2", "us-east-1", "eu-west-1"]:
    raise Exception("This notebook uses RoboMaker which is available only in US East (N. Virginia),"
                    "US West (Oregon) and EU (Ireland). Please switch to one of these regions.")

### S3 버킷 설정

체크포인트(checkpoint) 및 메타데이터에 사용하려는 S3 버킷에 대한 연결 및 인증을 설정합니다.

In [ ]:
# S3 bucket
s3_bucket = sage_session.default_bucket()

# SDK appends the job name and output folder
s3_output_path = 's3://{}/'.format(s3_bucket)

#Ensure that the S3 prefix contains the keyword 'sagemaker'
s3_prefix = job_name_prefix + "-sagemaker-" + strftime("%y%m%d-%H%M%S", gmtime())

# Get the AWS account id of this account
sts = boto3.client("sts")
account_id = sts.get_caller_identity()['Account']

print("Using s3 bucket {}".format(s3_bucket))
print("Model checkpoints and other metadata will be stored at: \ns3://{}/{}".format(s3_bucket, s3_prefix))

### IAM 역할 생성

SageMaker 노트북 `role = sagemaker.get_execution_role()`을 실행할 때 실행 역할(execution role)을 얻거나 로컬 시스템에서 실행할 때 utils 메소드 `role = get_execution_role()`을 사용하여 실행 역할을 작성하세요.

In [ ]:
try:
    sagemaker_role = sagemaker.get_execution_role()
except:
    sagemaker_role = get_execution_role('sagemaker')

print("Using Sagemaker IAM role arn: \n{}".format(sagemaker_role))

> 시뮬레이터는 AWS RoboMaker 서비스를 기반으로 하므로, 이 노트북은 `SageMaker 로컬 모드`에서 실행할 수 없습니다.

### 이 노트북에서 AWS RoboMaker를 호출하기 위한 권한 설정

이 노트북이 AWS RoboMaker 작업을 실행할 수 있게 하려면, 이 노트북의 기본 실행 역할에 하나의 신뢰 관계(trust relationshop)를 추가해야 합니다.

In [1]:
display(Markdown(generate_help_for_robomaker_trust_relationship(sagemaker_role)))

NameError: name 'Markdown' is not defined

### SageMaker에서 S3 버킷으로의 권한 설정 

SageMaker는 Redis IP 주소 모델을 S3 버킷에 씁니다. 따라서, 버킷에 대한 `PutObject` 권한이 필요합니다. 이 권한으로 사용중인 sagemaker 역할(role)을 확인하세요.

In [ ]:
display(Markdown(generate_s3_write_permission_for_sagemaker_role(sagemaker_role)))


### SageMaker가 KinesisVideoStream을 생성할 수 있는 권한 설정 

SageMaker 노트북은 kinesis 비디오 스트리머를 생성해야 합니다. kinesis 비디오 스트리머에서 자동차가 에피소드를 생성하는 것을 관찰할 수 있습니다.

In [ ]:
display(Markdown(generate_kinesis_create_permission_for_sagemaker_role(sagemaker_role)))

### 도커 이미지 빌드 및 푸시

./Dockerfile 파일은 docker에 설치된 모든 패키지를 포함합니다. 기본 sagemaker 컨테이너를 사용하는 대신 이 도커 컨테이너를 사용할 것입니다.

In [ ]:
%%time
from copy_to_sagemaker_container import get_sagemaker_docker, copy_to_sagemaker_container, get_custom_image_name
cpu_or_gpu = 'gpu' if instance_type.startswith('ml.p') else 'cpu'
repository_short_name = "sagemaker-docker-%s" % cpu_or_gpu
custom_image_name = get_custom_image_name(repository_short_name)
try:
    print("Copying files from your notebook to existing sagemaker container")
    sagemaker_docker_id = get_sagemaker_docker(repository_short_name)
    copy_to_sagemaker_container(sagemaker_docker_id, repository_short_name)
except Exception as e:
    print("Creating sagemaker container")
    docker_build_args = {
        'CPU_OR_GPU': cpu_or_gpu, 
        'AWS_REGION': boto3.Session().region_name,
    }
    custom_image_name = build_and_push_docker_image(repository_short_name, build_args=docker_build_args)
    print("Using ECR image %s" % custom_image_name)

### 도커 이미지 제거

Docker를 완전히 제거하거나 SageMaker 인스턴스의 공간을 정리하려는 경우에만 제거하세요.

In [ ]:
# !docker rm -f $(docker ps -a -q);
# !docker rmi -f $(docker images -q);

### VPC 구성 

SageMaker와 RoboMaker는 네트워크를 통해 서로 통신해야 하므로, 이러한 서비스는 모두 VPC 모드에서 실행해야 합니다. 작업 시작 스크립트에 서브넷과 보안 그룹을 제공하면 됩니다. 
deepracer-vpc 스택이 생성되었는지 확인하고 존재하는 경우 이를 사용합니다. (AWS Deepracer 콘솔을 한 번 이상 사용하여 모델을 생성하는 경우에 존재 합니다). 그렇지 않으면 기본 VPC 스택을 사용합니다.

In [ ]:
ec2 = boto3.client('ec2')

#
# Check if the user has Deepracer-VPC and use that if its present. This will have all permission.
# This VPC will be created when you have used the Deepracer console and created one model atleast
# If this is not present. Use the default VPC connnection
#
deepracer_security_groups = [group["GroupId"] for group in ec2.describe_security_groups()['SecurityGroups']\
                             if group['GroupName'].startswith("aws-deepracer-")]

# deepracer_security_groups = False
if(deepracer_security_groups):
    print("Using the DeepRacer VPC stacks. This will be created if you run one training job from console.")
    deepracer_vpc = [vpc['VpcId'] for vpc in ec2.describe_vpcs()['Vpcs'] \
                     if "Tags" in vpc for val in vpc['Tags'] \
                     if val['Value'] == 'deepracer-vpc'][0]
    deepracer_subnets = [subnet["SubnetId"] for subnet in ec2.describe_subnets()["Subnets"] \
                         if subnet["VpcId"] == deepracer_vpc]
else:
    print("Using the default VPC stacks")
    deepracer_vpc = [vpc['VpcId'] for vpc in ec2.describe_vpcs()['Vpcs'] if vpc["IsDefault"] == True][0]

    deepracer_security_groups = [group["GroupId"] for group in ec2.describe_security_groups()['SecurityGroups'] \
                                 if 'VpcId' in group and group["GroupName"] == "default" and group["VpcId"] == deepracer_vpc]

    deepracer_subnets = [subnet["SubnetId"] for subnet in ec2.describe_subnets()["Subnets"] \
                         if subnet["VpcId"] == deepracer_vpc and subnet['DefaultForAz']==True]

print("Using VPC:", deepracer_vpc)
print("Using security group:", deepracer_security_groups)
print("Using subnets:", deepracer_subnets)

### 라우트 테이블 생성

VPC 모드에서 실행 중인 SageMaker 작업은 S3 자원에 액세스할 수 없습니다. 따라서, SageMaker 컨테이너에서 S3에 액세스할 수 있도록 VPC S3 엔드포인트를 생성해야 합니다. VPC 모드에 대한 자세한 내용을 보려면, [이 링크](https://docs.aws.amazon.com/sagemaker/latest/dg/train-vpc.html)를 방문하세요.

In [ ]:
#TODO: Explain to customer what CREATE_ROUTE_TABLE is doing
CREATE_ROUTE_TABLE = True

def create_vpc_endpoint_table():
    print("Creating ")
    try:
        route_tables = [route_table["RouteTableId"] for route_table in ec2.describe_route_tables()['RouteTables']\
                        if route_table['VpcId'] == deepracer_vpc]
    except Exception as e:
        if "UnauthorizedOperation" in str(e):
            display(Markdown(generate_help_for_s3_endpoint_permissions(sagemaker_role)))
        else:
            display(Markdown(create_s3_endpoint_manually(aws_region, deepracer_vpc)))
        raise e

    print("Trying to attach S3 endpoints to the following route tables:", route_tables)
    
    if not route_tables:
        raise Exception(("No route tables were found. Please follow the VPC S3 endpoint creation "
                         "guide by clicking the above link."))
    try:
        ec2.create_vpc_endpoint(DryRun=False,
                                VpcEndpointType="Gateway",
                                VpcId=deepracer_vpc,
                                ServiceName="com.amazonaws.{}.s3".format(aws_region),
                                RouteTableIds=route_tables)
        print("S3 endpoint created successfully!")
    except Exception as e:
        if "RouteAlreadyExists" in str(e):
            print("S3 endpoint already exists.")
        elif "UnauthorizedOperation" in str(e):
            display(Markdown(generate_help_for_s3_endpoint_permissions(role)))
            raise e
        else:
            display(Markdown(create_s3_endpoint_manually(aws_region, deepracer_vpc)))
            raise e

if CREATE_ROUTE_TABLE:
    create_vpc_endpoint_table()

## 환경 설정

환경은 "deepracer_racetrack_env.py"라는 파이썬 파일에 정의되어 있으며, 파일은 `src/markov/environments/`에서 찾을 수 있습니다. 이 파일은 Gazebo 기반 RoboMakersimulator의 gym 인터페이스를 구현합니다. 이것은 SageMaker와 RoboMaker에서 사용하는 공통 환경 파일입니다. 환경 변수-`NODE_TYPE`은 코드가 실행 중인 노드를 정의합니다. 따라서 `rospy` 의존성이 있는 표현식은 RoboMaker에서만 실행됩니다.

`src/markov/rewards/`에서 `reward_function`을 수정하여 다양한 보상 함수를 실험할 수 있습니다. 또한, `src/markov/actions/`.json 파일을 수정하여 행동 공간(action space)과 조향 각도(steering angles)를 변경할 수 있습니다

### RL 알고리즘에 대한 사전 설정 구성

RL 훈련 작업을 구성하는 파라미터는 `src/markov/presets/`에 정의되어 있습니다. 사전 설정 파일을 사용하여 에이전트 파라메터들을 정의하여 특정 에이전트 알고리즘을 선택할 수 있습니다. 이 예에서는 Clipped PPO를 사용하는 것이 좋습니다. 이 파일을 편집하여 학습률(learning_rate), 신경망 구조, 배치 크기(batch_size), 감가율(discount factor) 등과 같은 알고리즘 매개 변수를 수정할 수 있습니다.

In [ ]:
# Uncomment the pygmentize code lines to see the code

# Reward function
#!pygmentize src/markov/rewards/default.py

# Action space
#!pygmentize src/markov/actions/single_speed_stereo_shallow.json

# Preset File
#!pygmentize src/markov/presets/default.py
#!pygmentize src/markov/presets/preset_attention_layer.py

### SageMaker 및 RoboMaker가 선택할 수 있도록 사용자 정의 파일을 S3 버킷으로 복사

In [ ]:
s3_location = "s3://%s/%s" % (s3_bucket, s3_prefix)
print(s3_location)

# Clean up the previously uploaded files
!aws s3 rm --recursive {s3_location}

!aws s3 cp src/markov/rewards/default.py {s3_location}/customer_reward_function.py

!aws s3 cp src/markov/actions/default.json {s3_location}/model/model_metadata.json

#!aws s3 cp src/markov/presets/default.py {s3_location}/presets/preset.py
#!aws s3 cp src/markov/presets/preset_attention_layer.py {s3_location}/presets/preset.py

### Python SDK / 스크립트 모드를 사용하여 RL 모델 훈련

다음으로, 훈련 진행 상황을 모니터링하기 위해, cloudwatch 로그에서 캡처할 알고리즘 지표들을 아래와 같이 정의합니다. 이들은 알고리즘에 특화된 파라메터들이며 알고리즘마다 다를 수 있습니다. 이 예에서는 [Clipped PPO](https://coach.nervanasys.com/algorithms/policy_optimization/cppo/index.html)를 사용합니다.

In [ ]:
metric_definitions = [
    # Training> Name=main_level/agent, Worker=0, Episode=19, Total reward=-102.88, Steps=19019, Training iteration=1
    {'Name': 'reward-training',
     'Regex': '^Training>.*Total reward=(.*?),'},
    
    # Policy training> Surrogate loss=-0.32664725184440613, KL divergence=7.255815035023261e-06, Entropy=2.83156156539917, training epoch=0, learning_rate=0.00025
    {'Name': 'ppo-surrogate-loss',
     'Regex': '^Policy training>.*Surrogate loss=(.*?),'},
     {'Name': 'ppo-entropy',
     'Regex': '^Policy training>.*Entropy=(.*?),'},
   
    # Testing> Name=main_level/agent, Worker=0, Episode=19, Total reward=1359.12, Steps=20015, Training iteration=2
    {'Name': 'reward-testing',
     'Regex': '^Testing>.*Total reward=(.*?),'},
]

RL 작업을 훈련하기 위해 RLEstimator를 사용합니다.

1. 환경, 사전 설정 및 훈련 코드가 업로드되는 소스 디렉토리를 지정합니다.
2. 엔트리포인트를 훈련 코드로 지정합니다.
3. RL 툴킷 및 프레임워크 선택을 지정합니다. RL 컨테이너의 ECR 경로로 자동 확인 가능합니다.
4. 모델 체크포인트 및 메타데이터를 저장하기 위해 인스턴스 수, 인스턴스 유형, 작업 이름, s3_bucket 및 s3_prefix와 같은 학습 파라메터들을 정의합니다. **현재 1개의 훈련 인스턴스 만 지원됩니다.**
5. 이 예에서는 RLCOACH_PRESET을 "deepracer"로 설정합니다.
6. 로그에서 캡처하려는 지표를 정의합니다. CloudWatch 및 SageMaker Notebook에서도 시각화할 수 있습니다.

In [ ]:
estimator = RLEstimator(entry_point="training_worker.py",
                        source_dir='src',
                        image_name=custom_image_name,
                        dependencies=["common/"],
                        role=sagemaker_role,
                        train_instance_type=instance_type,
                        train_instance_count=1,
                        output_path=s3_output_path,
                        base_job_name=job_name_prefix,
                        metric_definitions=metric_definitions,
                        train_max_run=job_duration_in_seconds,
                        hyperparameters={
                            "s3_bucket": s3_bucket,
                            "s3_prefix": s3_prefix,
                            "aws_region": aws_region,
                            "model_metadata_s3_key": "%s/model/model_metadata.json" % s3_prefix,
                            "reward_function_s3_source": "%s/customer_reward_function.py" % s3_prefix,
                            "batch_size": "64",
                            "num_epochs": "10",
                            "stack_size": "1",
                            "lr": "0.0003",
                            "exploration_type": "Categorical",
                            "e_greedy_value": "1",
                            "epsilon_steps": "10000",
                            "beta_entropy": "0.01",
                            "discount_factor": "0.999",
                            "loss_type": "Huber",
                            "num_episodes_between_training": "20"
                        },
                        subnets=deepracer_subnets,
                        security_group_ids=deepracer_security_groups,
                    )

estimator.fit(wait=False)
job_name = estimator.latest_training_job.job_name
print("Training job: %s" % job_name)

In [ ]:
training_job_arn = estimator.latest_training_job.describe()['TrainingJobArn']

### Kinesis 비디오 스트림 생성

In [ ]:
kvs_stream_name = "dr-kvs-{}".format(job_name)

!aws --region {aws_region} kinesisvideo create-stream --stream-name {kvs_stream_name} --media-type video/h264 --data-retention-in-hours 24
print ("Created kinesis video stream {}".format(kvs_stream_name))

### RoboMaker 작업 시작

In [ ]:
robomaker = boto3.client("robomaker")

### Simulation 어플리케이션 생성

In [ ]:
robomaker_s3_key = 'robomaker/simulation_ws.tar.gz'
robomaker_source = {'s3Bucket': s3_bucket,
                    's3Key': robomaker_s3_key,
                    'architecture': "X86_64"}
simulation_software_suite={'name': 'Gazebo',
                           'version': '7'}
robot_software_suite={'name': 'ROS',
                      'version': 'Kinetic'}
rendering_engine={'name': 'OGRE',
                  'version': '1.x'}

RoboMaker 서비스에서 제공하는 DeepRacer 번들을 다운로드하고 S3 버킷에 업로드하여 RoboMaker 시뮬레이션 애플리케이션을 작성하세요.

In [ ]:
if not os.path.exists('./build/output.tar.gz'):
    print("Using the latest simapp from public s3 bucket")
    # Download Robomaker simApp for the deepracer public s3 bucket
    simulation_application_bundle_location = "s3://deepracer-managed-resources-us-east-1/deepracer-simapp.tar.gz"
    !aws s3 cp {simulation_application_bundle_location} ./

    # Remove if the Robomaker sim-app is present in s3 bucket
    !aws s3 rm s3://{s3_bucket}/{robomaker_s3_key}

    # Uploading the Robomaker SimApp to your S3 bucket
    !aws s3 cp ./deepracer-simapp.tar.gz s3://{s3_bucket}/{robomaker_s3_key}

    # Cleanup the locally downloaded version of SimApp
    !rm deepracer-simapp.tar.gz
else:
    print("Using the simapp from build directory")
    !aws s3 cp ./build/output.tar.gz s3://{s3_bucket}/{robomaker_s3_key}

In [ ]:
app_name = "deepracer-notebook-application" + strftime("%y%m%d-%H%M%S", gmtime())

print(app_name)
try:
    response = robomaker.create_simulation_application(name=app_name,
                                                       sources=[robomaker_source],
                                                       simulationSoftwareSuite=simulation_software_suite,
                                                       robotSoftwareSuite=robot_software_suite,
                                                       renderingEngine=rendering_engine)
    simulation_app_arn = response["arn"]
    print("Created a new simulation app with ARN:", simulation_app_arn)
except Exception as e:
    if "AccessDeniedException" in str(e):
        display(Markdown(generate_help_for_robomaker_all_permissions(role)))
        raise e
    else:
        raise e

### RoboMaker에서 시뮬레이션 작업 시작

환경을 시뮬레이션하고 훈련을 위해, 이 데이터를 SageMaker와 공유하는 [AWS RoboMaker](https://console.aws.amazon.com/robomaker/home#welcome) 시뮬레이션 작업을 생성합니다.

In [ ]:
s3_yaml_name="training_params.yaml"
world_name = "reInvent2019_track"

!touch {s3_yaml_name}
!echo "WORLD_NAME:                           \"{world_name}\"" | tee -a {s3_yaml_name}
!echo "SAGEMAKER_SHARED_S3_BUCKET:           \"{s3_bucket}\"" | tee -a {s3_yaml_name}
!echo "SAGEMAKER_SHARED_S3_PREFIX:           \"{s3_prefix}\"" | tee -a {s3_yaml_name}
!echo "TRAINING_JOB_ARN:                     \"{training_job_arn}\"" | tee -a {s3_yaml_name}
!echo "METRICS_S3_BUCKET:                    \"{s3_bucket}\"" | tee -a {s3_yaml_name}
!echo "METRICS_S3_OBJECT_KEY:                \"{s3_prefix}/training_metrics.json\"" | tee -a {s3_yaml_name}
!echo "AWS_REGION:                           \"{aws_region}\"" | tee -a {s3_yaml_name}
!echo "TARGET_REWARD_SCORE:                  \"None\"" | tee -a {s3_yaml_name}
!echo "NUMBER_OF_EPISODES:                   \"0\"" | tee -a {s3_yaml_name}
!echo "ROBOMAKER_SIMULATION_JOB_ACCOUNT_ID:  \"{account_id}\"" | tee -a {s3_yaml_name}
!echo "JOB_TYPE:                             \"TRAINING\"" | tee -a {s3_yaml_name}
!echo "CHANGE_START_POSITION:                \"true\"" | tee -a {s3_yaml_name}
!echo "ALTERNATE_DRIVING_DIRECTION:          \"false\"" | tee -a {s3_yaml_name}
!echo "KINESIS_VIDEO_STREAM_NAME:            \"{kvs_stream_name}\"" | tee -a {s3_yaml_name}
!echo "REWARD_FILE_S3_KEY:                   \"{s3_prefix}/customer_reward_function.py\"" | tee -a {s3_yaml_name}
!echo "MODEL_METADATA_FILE_S3_KEY:           \"{s3_prefix}/model/model_metadata.json\"" | tee -a {s3_yaml_name}

!echo "NUMBER_OF_OBSTACLES:                  \"0\"" | tee -a {s3_yaml_name}
!echo "MIN_DISTANCE_BETWEEN_OBSTACLES:       \"2.0\"" | tee -a {s3_yaml_name}
!echo "RANDOMIZE_OBSTACLE_LOCATIONS:         \"false\"" | tee -a {s3_yaml_name}
!echo "PSEUDO_RANDOMIZE_OBSTACLE_LOCATIONS:  \"false\"" | tee -a {s3_yaml_name}
!echo "NUMBER_OF_PSEUDO_RANDOM_PLACEMENTS:   \"2\"" | tee -a {s3_yaml_name}
!echo "IS_OBSTACLE_BOT_CAR:                  \"false\"" | tee -a {s3_yaml_name}   

!echo "IS_LANE_CHANGE:                       \"false\"" | tee -a {s3_yaml_name}
!echo "LOWER_LANE_CHANGE_TIME:               \"3.0\"" | tee -a {s3_yaml_name}
!echo "UPPER_LANE_CHANGE_TIME:               \"5.0\"" | tee -a {s3_yaml_name}
!echo "LANE_CHANGE_DISTANCE:                 \"1.0\"" | tee -a {s3_yaml_name}

!echo "NUMBER_OF_BOT_CARS:                   \"6\"" | tee -a {s3_yaml_name}
!echo "MIN_DISTANCE_BETWEEN_BOT_CARS:        \"2.0\"" | tee -a {s3_yaml_name}
!echo "RANDOMIZE_BOT_CAR_LOCATIONS:          \"false\"" | tee -a {s3_yaml_name}
!echo "BOT_CAR_SPEED:                        \"0.2\"" | tee -a {s3_yaml_name}

!echo "CAR_COLOR:                            \"LightBlue\"" | tee -a {s3_yaml_name}
!echo "FIRST_PERSON_VIEW:                    \"false\"" | tee -a {s3_yaml_name}
!echo "CHANGE_START_POSITION:                \"true\"" | tee -a {s3_yaml_name}

print("Upload yaml settings to S3")
!aws s3 cp ./training_params.yaml {s3_location}/training_params.yaml
!rm training_params.yaml

In [ ]:
num_simulation_workers = 1

envriron_vars = {
    "S3_YAML_NAME": s3_yaml_name,
    "SAGEMAKER_SHARED_S3_PREFIX": s3_prefix,
    "SAGEMAKER_SHARED_S3_BUCKET": s3_bucket,
    "WORLD_NAME": world_name,
    "KINESIS_VIDEO_STREAM_NAME": kvs_stream_name,
    "APP_REGION": aws_region,
    "MODEL_METADATA_FILE_S3_KEY": "%s/model/model_metadata.json" % s3_prefix
}

simulation_application = {"application":simulation_app_arn,
                          "launchConfig": {"packageName": "deepracer_simulation_environment",
                                           "launchFile": "distributed_training.launch",
                                           "environmentVariables": envriron_vars}
                         }


vpcConfig = {"subnets": deepracer_subnets,
             "securityGroups": deepracer_security_groups,
             "assignPublicIp": True}

responses = []
for job_no in range(num_simulation_workers):
    client_request_token = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    response =  robomaker.create_simulation_job(iamRole=sagemaker_role,
                                            clientRequestToken=client_request_token,
                                            maxJobDurationInSeconds=job_duration_in_seconds,
                                            failureBehavior="Continue",
                                            simulationApplications=[simulation_application],
                                            vpcConfig=vpcConfig
                                            )
    responses.append(response)

print("Created the following jobs:")
job_arns = [response["arn"] for response in responses]
for response in responses:
    print("Job ARN", response["arn"]) 

### RoboMaker에서 시뮬레이션 시각화
RoboMaker 콘솔을 방문하여 시뮬레이션을 시각화하거나 다음 셀을 실행하여 하이퍼링크를 생성할 수 있습니다.

In [ ]:
display(Markdown(generate_robomaker_links(job_arns, aws_region)))

### 임시 폴더 생성

In [ ]:
tmp_dir = "/tmp/{}".format(job_name)
os.system("mkdir {}".format(tmp_dir))
print("Create local folder {}".format(tmp_dir))

### 훈련 작업의 지표 plot

In [ ]:
%matplotlib inline
import pandas as pd
import json

training_metrics_file = "training_metrics.json"
training_metrics_path = "{}/{}".format(s3_bucket, training_metrics_file)
wait_for_s3_object(s3_bucket, training_metrics_path, tmp_dir)

json_file = "{}/{}".format(tmp_dir, training_metrics_file)
with open(json_file) as fp:  
    data = json.load(fp)

df = pd.DataFrame(data['metrics'])
x_axis = 'episode'
y_axis = 'reward_score'

plt = df.plot(x=x_axis,y=y_axis, figsize=(12,5), legend=True, style='b-')
plt.set_ylabel(y_axis);
plt.set_xlabel(x_axis);

### RoboMaker 및 SageMaker 훈련 작업 삭제

RoboMaker 및 SageMaker 작업을 종료하려면 아래 셀을 실행하세요

In [ ]:
# # Cancelling robomaker job
# for job_arn in job_arns:
#     robomaker.cancel_simulation_job(job=job_arn)

# # Stopping sagemaker training job
# sage_session.sagemaker_client.stop_training_job(TrainingJobName=estimator._current_job_name)

### 평가 - ReInvent 트랙

In [ ]:
s3_yaml_name="evaluation_params.yaml"
world_name = "reInvent2019_track"

!touch {s3_yaml_name}
echo "WORLD_NAME:                           \"{world_name}\"" | tee {s3_yaml_name}
echo "MODEL_S3_BUCKET:                      \"{s3_bucket}\"" | tee -a {s3_yaml_name}
echo "MODEL_S3_PREFIX:                      \"{s3_prefix}\"" | tee -a {s3_yaml_name}
echo "AWS_REGION:                           \"{aws_region}\"" | tee -a {s3_yaml_name}
echo "METRICS_S3_BUCKET:                    \"{s3_bucket}\"" | tee -a {s3_yaml_name}
echo "METRICS_S3_OBJECT_KEY:                \"{s3_prefix}/evaluation_metrics.json\"" | tee -a {s3_yaml_name}
echo "NUMBER_OF_TRIALS:                     \"10\"" | tee -a {s3_yaml_name}
echo "ROBOMAKER_SIMULATION_JOB_ACCOUNT_ID:  \"{account_id}\"" | tee -a {s3_yaml_name}
echo "JOB_TYPE:                             \"EVALUATION\"" | tee -a {s3_yaml_name}

echo "NUMBER_OF_OBSTACLES:                  \"0\"" | tee -a {s3_yaml_name}
echo "MIN_DISTANCE_BETWEEN_OBSTACLES:       \"2.0\"" | tee -a {s3_yaml_name}
echo "RANDOMIZE_OBSTACLE_LOCATIONS:         \"false\"" | tee -a {s3_yaml_name}
echo "PSEUDO_RANDOMIZE_OBSTACLE_LOCATIONS:  \"false\"" | tee -a {s3_yaml_name}
echo "NUMBER_OF_PSEUDO_RANDOM_PLACEMENTS:   \"2\"" | tee -a {s3_yaml_name}
echo "IS_OBSTACLE_BOT_CAR:                  \"false\"" | tee -a {s3_yaml_name}  

echo "IS_LANE_CHANGE:                       \"false\"" | tee -a {s3_yaml_name}
echo "LOWER_LANE_CHANGE_TIME:               \"3.0\"" | tee -a {s3_yaml_name}
echo "UPPER_LANE_CHANGE_TIME:               \"5.0\"" | tee -a {s3_yaml_name}
echo "LANE_CHANGE_DISTANCE:                 \"1.0\"" | tee -a {s3_yaml_name}

echo "NUMBER_OF_BOT_CARS:                   \"6\"" | tee -a {s3_yaml_name}
echo "MIN_DISTANCE_BETWEEN_BOT_CARS:        \"2.0\"" | tee -a {s3_yaml_name}
echo "RANDOMIZE_BOT_CAR_LOCATIONS:          \"true\"" | tee -a {s3_yaml_name}
echo "BOT_CAR_SPEED:                        \"0.2\"" | tee -a {s3_yaml_name}
echo "CAR_COLOR:                            \"LightBlue\"" | tee -a {s3_yaml_name}

!echo "CAR_COLOR:                           \"LightBlue\"" | tee -a {s3_yaml_name}
!echo "FIRST_PERSON_VIEW:                   \"false\"" | tee -a {s3_yaml_name}
!echo "CHANGE_START_POSITION:               \"true\"" | tee -a {s3_yaml_name}

print("Upload yaml settings to S3")
!aws s3 cp ./evaluation_params.yaml {s3_location}/evaluation_params.json

In [ ]:
sys.path.append("./src")

num_simulation_workers = 1

envriron_vars = {
    "S3_YAML_NAME": s3_yaml_name,
    "MODEL_S3_PREFIX": s3_prefix,
    "MODEL_S3_BUCKET": s3_bucket,
    "WORLD_NAME": world_name,
    "KINESIS_VIDEO_STREAM_NAME": kvs_stream_name,
    "APP_REGION": aws_region,
    "MODEL_METADATA_FILE_S3_KEY": "%s/model/model_metadata.json" % s3_prefix
}

simulation_application = {
    "application":simulation_app_arn,
    "launchConfig": {
         "packageName": "deepracer_simulation_environment",
         "launchFile": "evaluation.launch",
         "environmentVariables": envriron_vars
    }
}
                            
vpcConfig = {"subnets": deepracer_subnets,
             "securityGroups": deepracer_security_groups,
             "assignPublicIp": True}

responses = []
for job_no in range(num_simulation_workers):
    response =  robomaker.create_simulation_job(clientRequestToken=strftime("%Y-%m-%d-%H-%M-%S", gmtime()),
                                                outputLocation={ 
                                                  "s3Bucket": s3_bucket,
                                                  "s3Prefix": s3_prefix
                                                },
                                                maxJobDurationInSeconds=job_duration_in_seconds,
                                                iamRole=sagemaker_role,
                                                failureBehavior="Continue",
                                                simulationApplications=[simulation_application],
                                                vpcConfig=vpcConfig)
    responses.append(response)

# print("Created the following jobs:")
for response in responses:
    print("Job ARN", response["arn"]) 

### 임시 폴더 생성

In [ ]:
evaluation_metrics_file = "evaluation_metrics.json"
evaluation_metrics_path = "{}/{}".format(s3_bucket, evaluation_metrics_file)
wait_for_s3_object(s3_bucket, evaluation_metrics_path, tmp_dir)

json_file = "{}/{}".format(tmp_dir, evaluation_metrics_file)
with open(json_file) as fp:  
    data = json.load(fp)

df = pd.DataFrame(data['metrics'])
# Converting milliseconds to seconds
df['elapsed_time'] = df['elapsed_time_in_milliseconds']/1000
df = df[['trial', 'completion_percentage', 'elapsed_time']]

display(df)

### 시뮬레이션 어플리케이션 리소스 정리

In [ ]:
# robomaker.delete_simulation_application(application=simulation_app_arn)

### S3 버킷 삭제 (원하는 경우 아래 코드 셀의 awscli 명령들을 주석 해제하세요.)

In [ ]:
## Uncomment if you only want to clean the s3 bucket
# sagemaker_s3_folder = "s3://{}/{}".format(s3_bucket, s3_prefix)
# !aws s3 rm --recursive {sagemaker_s3_folder}

# robomaker_s3_folder = "s3://{}/{}".format(s3_bucket, job_name)
# !aws s3 rm --recursive {robomaker_s3_folder}

# robomaker_sim_app = "s3://{}/{}".format(s3_bucket, 'robomaker')
# !aws s3 rm --recursive {robomaker_sim_app}

# model_output = "s3://{}/{}".format(s3_bucket, s3_bucket)
# !aws s3 rm --recursive {model_output}